<a href="https://colab.research.google.com/github/ZhafranR/Coursera_Capstone/blob/master/Final_Project_Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import folium
from requests import get
from bs4 import BeautifulSoup 
import requests 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import DBSCAN 
import seaborn as sns
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.preprocessing import StandardScaler 

In [280]:
path = '/Jaksel Latlong.csv'
df = pd.read_csv(path)
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Cilandak,Cilandak Barat,-6.281769,106.792179
1,Cilandak,Cipete Selatan,-6.274093,106.798790
2,Cilandak,Gandaria Selatan,-6.270547,106.793827
3,Cilandak,Lebak Bulus,-6.304664,106.784375
4,Cilandak,Pondok Labu,-6.306311,106.798577


In [0]:
lat_jaksel = -6.268991
long_jaksel = 106.806039

In [282]:
map_jaksel = folium.Map(location=[lat_jaksel, long_jaksel], zoom_start=10)

for i, j, Neighborhood, Borough in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Borough']):
  label = '{}, {}'.format(Neighborhood, Borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
  [i, j],
  radius=5,
  popup=label,
  color='blue',
  fill=True,
  fill_color='#3186cc',
  fill_opacity=0.7,
  parse_html=False).add_to(map_jaksel)  

map_jaksel

In [283]:
CLIENT_ID = 'HFBBZVW1NTKDX1S23BAUD23OGUHC5MYYXGOQQUNU3W3OETBO' # your Foursquare ID
CLIENT_SECRET = 'D2HYVX1AFCWU2X4VEVA32WMC3W4WUEJWFSCUFEAWFIWZN4VQ' # your Foursquare Secret
VERSION = '20202105' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HFBBZVW1NTKDX1S23BAUD23OGUHC5MYYXGOQQUNU3W3OETBO
CLIENT_SECRET:D2HYVX1AFCWU2X4VEVA32WMC3W4WUEJWFSCUFEAWFIWZN4VQ


In [284]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cilandak Barat are -6.281769000000001, 106.792179.


In [285]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=HFBBZVW1NTKDX1S23BAUD23OGUHC5MYYXGOQQUNU3W3OETBO&client_secret=D2HYVX1AFCWU2X4VEVA32WMC3W4WUEJWFSCUFEAWFIWZN4VQ&v=20202105&ll=-6.281769000000001,106.792179&radius=500&limit=100'

In [286]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec6a40740a7ea0028304cf8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5166c1c3e4b0ef8110ab1ea9-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'primary': True,
         'shortName': 'Coffee Shop'}],
       'id': '5166c1c3e4b0ef8110ab1ea9',
       'location': {'address': 'Jl. Terogong Raya Kav. 28',
        'cc': 'ID',
        'city': 'Jakarta',
        'country': 'Indonesia',
        'crossStreet': 'Jalan Terogong 3',
        'distance': 123,
        'formattedAddress': ['Jl. Terogong Raya Kav. 28 (Jalan Terogong 3)',
         'Jakarta Selatan',
         'Jakar

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [288]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Starbucks,Coffee Shop,-6.282600,106.791437
1,Hero,Supermarket,-6.282402,106.790890
2,Papaya Fresh Gallery,Grocery Store,-6.281455,106.794638
3,Satu Lagi Bar,Bar,-6.282407,106.790896
4,Woodpecker Coffee,Coffee Shop,-6.284315,106.788571
5,ACE Hardware,Hardware Store,-6.283973,106.796007
6,Lifespa Fitness,Gym,-6.283856,106.791328
7,TITAN (Baking Ingredients & Tools),Bakery,-6.282845,106.795983
8,Bubur Ayam Setia Mitra,Breakfast Spot,-6.281744,106.796219
9,Hotel Kristal,Hotel,-6.283829,106.791196


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [290]:
jaksel_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Cilandak Barat
Cipete Selatan
Gandaria Selatan
Lebak Bulus
Pondok Labu
Ciganjur
Cipedak
Jagakarsa
Lenteng Agung
Srengseng Sawah
Tanjung Barat
Cipete Utara
Gandaria Utara
Gunung
Kramat Pela
Melawai
Petogogan
Pulo
Rawa Barat
Selong 
Senayan
Cipulir
Grogol Selatan
Grogol Utara
Kebayoran Lama Selatan
Kebayoran Lama Utara
Pondok Pinang
Bangka
Kuningan Barat
Mampang Prapatan
Pela Mampang
Tegal Parang
Cikoko
Duren Tiga
Kalibata
Pancoran
Pengadegan
Rajawati
Cilandak Timur
Jati Padang
Kebagusan
Pasar Minggu
Pejaten Barat
Pejaten Timur
Ragunan
Bintaro
Pesanggrahan
Petakungan Selatan
Petakungan Utara
Ulujami
Guntur
Karet Kuningan
Karet Semanggi
Karet
Kuningan Timur
Menteng Atas
Pasar Manggis
Setiabudi
Bukit Duri
Kebon Baru
Manggarai Selatan
Manggarai
Menteng Dalam
Tebet Barat
Tebet Timur


In [291]:
print(jaksel_venues.shape)
jaksel_venues.head()

(1098, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cilandak Barat,-6.281769,106.792179,Starbucks,-6.282600,106.791437,Coffee Shop
1,Cilandak Barat,-6.281769,106.792179,Hero,-6.282402,106.790890,Supermarket
2,Cilandak Barat,-6.281769,106.792179,Papaya Fresh Gallery,-6.281455,106.794638,Grocery Store
3,Cilandak Barat,-6.281769,106.792179,Satu Lagi Bar,-6.282407,106.790896,Bar
4,Cilandak Barat,-6.281769,106.792179,Woodpecker Coffee,-6.284315,106.788571,Coffee Shop


In [292]:
jaksel_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bangka,9,9,9,9,9,9
Bintaro,12,12,12,12,12,12
Bukit Duri,19,19,19,19,19,19
Ciganjur,26,26,26,26,26,26
Cikoko,11,11,11,11,11,11
...,...,...,...,...,...,...
Tanjung Barat,7,7,7,7,7,7
Tebet Barat,40,40,40,40,40,40
Tebet Timur,41,41,41,41,41,41


In [293]:
print('There are {} uniques categories.'.format(len(jaksel_venues['Venue Category'].unique())))

There are 169 uniques categories.


In [294]:
# one hot encoding
jaksel_onehot = pd.get_dummies(jaksel_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jaksel_onehot['Neighborhood'] = jaksel_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jaksel_onehot.columns[-1]] + list(jaksel_onehot.columns[:-1])
jaksel_onehot = jaksel_onehot[fixed_columns]

jaksel_onehot.head()

,Women's Store,Acehnese Restaurant,African Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bar,Basketball Court,Beer Bar,Beer Garden,Betawinese Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Car Wash,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,...,Salad Place,Salon / Barbershop,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Sundanese Restaurant,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Theme Restaurant,Track,Train Station,Travel Agency,Turkish Restaurant,University,Vape Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [295]:
jaksel_grouped = jaksel_onehot.groupby('Neighborhood').mean().reset_index()
jaksel_grouped.nunique()

Neighborhood                     64
Women's Store                     3
Acehnese Restaurant               2
African Restaurant                2
American Restaurant               7
                                 ..
Vegetarian / Vegan Restaurant     4
Video Store                       2
Vietnamese Restaurant             5
Wine Bar                          4
Wings Joint                       2
Length: 169, dtype: int64

In [296]:
num_top_venues = 5

for hood in jaksel_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = jaksel_grouped[jaksel_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bangka----
               venue  freq
0        Coffee Shop  0.22
1         Restaurant  0.22
2        Art Gallery  0.11
3             Bistro  0.11
4  Convenience Store  0.11


----Bintaro----
                   venue  freq
0  Indonesian Restaurant  0.33
1             Food Truck  0.17
2             Restaurant  0.08
3      Fish & Chips Shop  0.08
4        Motorcycle Shop  0.08


----Bukit Duri----
                   venue  freq
0  Indonesian Restaurant  0.21
1                 Bakery  0.16
2             Donut Shop  0.11
3                   Café  0.05
4   Fast Food Restaurant  0.05


----Ciganjur----
                   venue  freq
0       Asian Restaurant  0.12
1            Coffee Shop  0.12
2      Convenience Store  0.08
3  Indonesian Restaurant  0.08
4         Ice Cream Shop  0.04


----Cikoko----
                   venue  freq
0            Coffee Shop  0.18
1            Gas Station  0.09
2  Indonesian Restaurant  0.09
3       Asian Restaurant  0.09
4     Chinese Restaurant  0.09


--

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [298]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = jaksel_grouped['Neighborhood']

for ind in np.arange(jaksel_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(jaksel_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
jaksel_merged = pd.merge(df, neighborhoods_venues_sorted, on='Neighborhood')
jaksel_merged.head()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cilandak,Cilandak Barat,-6.281769,106.792179,Coffee Shop,Asian Restaurant,Indonesian Restaurant,Convenience Store,Sports Bar,Bakery,Food Truck,Motorcycle Shop,Breakfast Spot,Music Venue
1,Cilandak,Cipete Selatan,-6.274093,106.798790,Fast Food Restaurant,Padangnese Restaurant,Food Truck,Indonesian Restaurant,Bakery,Motorcycle Shop,Breakfast Spot,Snack Place,Bookstore,Fried Chicken Joint
2,Cilandak,Gandaria Selatan,-6.270547,106.793827,Indonesian Restaurant,Café,Food Truck,Motorcycle Shop,Multiplex,Convenience Store,Farmers Market,Food Court,Food & Drink Shop,Flea Market
3,Cilandak,Lebak Bulus,-6.304664,106.784375,Café,Sushi Restaurant,Bubble Tea Shop,Hobby Shop,Paper / Office Supplies Store,Hardware Store,Asian Restaurant,Soccer Stadium,Coffee Shop,Shabu-Shabu Restaurant
4,Cilandak,Pondok Labu,-6.306311,106.798577,Movie Theater,Coffee Shop,Bubble Tea Shop,Indonesian Restaurant,Dumpling Restaurant,Grocery Store,Café,Shopping Mall,Farm,Food & Drink Shop


In [299]:
#clusters = []
jaksel_grouped_clustering = jaksel_grouped.drop('Neighborhood', 1)

'''
epsilon = 0.3
minimumSamples = 9
db = DBSCAN(eps=epsilon, min_samples=minimumSamples, algorithm='kd_tree').fit(jaksel_grouped_clustering)
labels = db.labels_
labels

# Firts, create an array of booleans using the labels from db.
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
core_samples_mask

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters_

# Remove repetition in labels by turning it into a set.
unique_labels = set(labels)
unique_labels
'''

# run k-means clustering
kclusters = 7
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jaksel_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

'''
for i in range(1,21):
  km = KMeans(n_clusters=i).fit(jaksel_grouped_clustering)
  clusters.append(km.inertia_)
'''

'\nfor i in range(1,21):\n  km = KMeans(n_clusters=i).fit(jaksel_grouped_clustering)\n  clusters.append(km.inertia_)\n'

In [300]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
jaksel_merged = df.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

jaksel_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cilandak,Cilandak Barat,-6.281769,106.792179,4,Coffee Shop,Asian Restaurant,Indonesian Restaurant,Convenience Store,Sports Bar,Bakery,Food Truck,Motorcycle Shop,Breakfast Spot,Music Venue
1,Cilandak,Cipete Selatan,-6.274093,106.798790,0,Fast Food Restaurant,Padangnese Restaurant,Food Truck,Indonesian Restaurant,Bakery,Motorcycle Shop,Breakfast Spot,Snack Place,Bookstore,Fried Chicken Joint
2,Cilandak,Gandaria Selatan,-6.270547,106.793827,0,Indonesian Restaurant,Café,Food Truck,Motorcycle Shop,Multiplex,Convenience Store,Farmers Market,Food Court,Food & Drink Shop,Flea Market
3,Cilandak,Lebak Bulus,-6.304664,106.784375,4,Café,Sushi Restaurant,Bubble Tea Shop,Hobby Shop,Paper / Office Supplies Store,Hardware Store,Asian Restaurant,Soccer Stadium,Coffee Shop,Shabu-Shabu Restaurant
4,Cilandak,Pondok Labu,-6.306311,106.798577,4,Movie Theater,Coffee Shop,Bubble Tea Shop,Indonesian Restaurant,Dumpling Restaurant,Grocery Store,Café,Shopping Mall,Farm,Food & Drink Shop


In [301]:
lat_jaksel = -6.268991
long_jaksel = 106.806039
map_jaksel = folium.Map(location=[lat_jaksel, long_jaksel], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jaksel_merged['Latitude'], jaksel_merged['Longitude'], jaksel_merged['Neighborhood'], jaksel_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_jaksel)
       
map_jaksel

In [302]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 0, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]].head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Cipete Selatan,Fast Food Restaurant,Padangnese Restaurant,Food Truck,Indonesian Restaurant,Bakery,Motorcycle Shop,Breakfast Spot,Snack Place,Bookstore,Fried Chicken Joint
2,Gandaria Selatan,Indonesian Restaurant,Café,Food Truck,Motorcycle Shop,Multiplex,Convenience Store,Farmers Market,Food Court,Food & Drink Shop,Flea Market
7,Jagakarsa,Indonesian Restaurant,Asian Restaurant,Department Store,BBQ Joint,Soccer Stadium,Café,Chinese Restaurant,Noodle House,Food Court,Arcade
8,Lenteng Agung,Indonesian Restaurant,Food Court,Noodle House,Motorcycle Shop,Soccer Stadium,Flea Market,Arcade,Asian Restaurant,Train Station,Café
10,Tanjung Barat,Indonesian Restaurant,Department Store,Arcade,Train Station,Donut Shop,Javanese Restaurant,Diner,Farm,Food Court,Food & Drink Shop


In [303]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 1, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]].head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Manggarai Selatan,Clothing Store,Wings Joint,Falafel Restaurant,Food Truck,Food Court,Food & Drink Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [304]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 2, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]].head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Grogol Selatan,Pharmacy,Convenience Store,Soup Place,Soccer Field,Noodle House,Electronics Store,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
34,Kalibata,Hotel,Fast Food Restaurant,Wings Joint,Electronics Store,Food Court,Food & Drink Shop,Flea Market,Fish & Chips Shop,Farmers Market,Farm
36,Pengadegan,Food Truck,Noodle House,Car Wash,BBQ Joint,Food Court,Food & Drink Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
42,Pejaten Barat,Food Truck,Bridal Shop,Pharmacy,Shopping Mall,Noodle House,Hospital,Hardware Store,Flea Market,Fast Food Restaurant,Farmers Market
46,Pesanggrahan,Fast Food Restaurant,Noodle House,Wings Joint,Falafel Restaurant,Food Court,Food & Drink Shop,Flea Market,Fish & Chips Shop,Farmers Market,Farm


In [305]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 3, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]].head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Srengseng Sawah,College Residence Hall,Wings Joint,Falafel Restaurant,Food Truck,Food Court,Food & Drink Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [306]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 4, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]].head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cilandak Barat,Coffee Shop,Asian Restaurant,Indonesian Restaurant,Convenience Store,Sports Bar,Bakery,Food Truck,Motorcycle Shop,Breakfast Spot,Music Venue
3,Lebak Bulus,Café,Sushi Restaurant,Bubble Tea Shop,Hobby Shop,Paper / Office Supplies Store,Hardware Store,Asian Restaurant,Soccer Stadium,Coffee Shop,Shabu-Shabu Restaurant
4,Pondok Labu,Movie Theater,Coffee Shop,Bubble Tea Shop,Indonesian Restaurant,Dumpling Restaurant,Grocery Store,Café,Shopping Mall,Farm,Food & Drink Shop
5,Ciganjur,Coffee Shop,Asian Restaurant,Indonesian Restaurant,Convenience Store,Sports Bar,Bakery,Food Truck,Motorcycle Shop,Breakfast Spot,Music Venue
6,Cipedak,Grocery Store,Department Store,Wings Joint,Food Court,Food & Drink Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [307]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 5, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]].head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,Pasar Manggis,Betawinese Restaurant,Hotel,Electronics Store,Flea Market,Market,Food Truck,Pizza Place,Dog Run,Donut Shop,Dumpling Restaurant


In [308]:
jaksel_merged.loc[jaksel_merged['Cluster Labels'] == 6, jaksel_merged.columns[[1] + list(range(5, jaksel_merged.shape[1]))]].head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Kebayoran Lama Selatan,Golf Course,Wings Joint,Falafel Restaurant,Food Court,Food & Drink Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
